In [21]:
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
import urllib3  # the lib that handles the url stuff
from PIL import Image
import numpy as np
import os
import csv

In [22]:
detector = hub.load("https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1")

In [23]:
def predict(my_image):
    # commented for open_image_from_google
    # my_image = image.smart_resize(my_image, size=(1024, 1024))
    x = image.img_to_array(my_image)
    x = np.expand_dims(x, axis=0)
    preds = detector(x)
    return preds

In [24]:
def open_image_from_google(coco):

    impath = "D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\964b69442edeec04.jpg"

    apple = Image.open(impath)
    result, x = predict(apple)
    results = {key:value.numpy() for key,value in result.items()}
    detection_classes = results['detection_classes']
    print("model thinks: " + coco[detection_classes[0][0]])

In [25]:
def get_coco_labels():
    http = urllib3.PoolManager()
    r = http.request('GET', 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt')
    dane = r.data.decode('UTF-8')
    my_data = dane.split('\n')

    coco = {i+1: my_data[i] for i in range(0,len(my_data))}
    return coco


In [26]:
def get_downloadable_labels_str(coco):
    d = {value:1 for value in coco.values()}
    # tmp = d["street sign"]

    # nowa googlowa          stara coco
    d["traffic sign"] = d.pop("street sign")
    d["bull"] = d.pop("cow")
    d["glasses"] = d.pop("eye glasses")
    d["boot"] = d.pop("shoe")
    d["hair dryer"] = d.pop("hair drier")
    d["doughnut"] = d.pop("donut")
    d["microwave oven"] = d.pop("microwave")
    d["telephone"] = d.pop("cell phone")
    d["computer keyboard"] = d.pop("keyboard")
    d["remote control"] = d.pop("remote")
    d["computer mouse"] = d.pop("mouse")
    d["television"] = d.pop("tv")
    d["Kitchen dining room table"] = d.pop("dining table")
    d["plant"] = d.pop("potted plant")
    d["ski"] = d.pop("skis")
    d["volleyball"] = d.pop("sports ball")
    d["coffee cup"] = d.pop("cup")
    d["flying disc"] = d.pop("frisbee")
    # hairbrush
    d.pop("hair brush")

    labels = list(d.keys())
    lb = [x.capitalize() for x in labels]
    lb1 = [label.replace(" ","_") for label in lb]
    str1 = " "
    names = str1.join(lb1)
    print(names)
    return names




Person Bicycle Car Motorcycle Airplane Bus Train Truck Boat Traffic_light Fire_hydrant Stop_sign Parking_meter Bench Bird Cat Dog Horse Sheep Elephant Bear Zebra Giraffe Hat Backpack Umbrella Handbag Tie Suitcase Snowboard Kite Baseball_bat Baseball_glove Skateboard Surfboard Tennis_racket Bottle Plate Wine_glass Fork Knife Spoon Bowl Banana Apple Sandwich Orange Broccoli Carrot Hot_dog Pizza Cake Chair Couch Bed Mirror Window Desk Toilet Door Laptop Oven Toaster Sink Refrigerator Blender Book Clock Vase Scissors Teddy_bear Toothbrush Traffic_sign Bull Glasses Boot Hair_dryer Doughnut Microwave_oven Telephone Computer_keyboard Remote_control Computer_mouse Television Kitchen_dining_room_table Plant Ski Volleyball Coffee_cup Flying_disc


'Person Bicycle Car Motorcycle Airplane Bus Train Truck Boat Traffic_light Fire_hydrant Stop_sign Parking_meter Bench Bird Cat Dog Horse Sheep Elephant Bear Zebra Giraffe Hat Backpack Umbrella Handbag Tie Suitcase Snowboard Kite Baseball_bat Baseball_glove Skateboard Surfboard Tennis_racket Bottle Plate Wine_glass Fork Knife Spoon Bowl Banana Apple Sandwich Orange Broccoli Carrot Hot_dog Pizza Cake Chair Couch Bed Mirror Window Desk Toilet Door Laptop Oven Toaster Sink Refrigerator Blender Book Clock Vase Scissors Teddy_bear Toothbrush Traffic_sign Bull Glasses Boot Hair_dryer Doughnut Microwave_oven Telephone Computer_keyboard Remote_control Computer_mouse Television Kitchen_dining_room_table Plant Ski Volleyball Coffee_cup Flying_disc'

In [27]:
def get_paths_to_photos_and_labels(dataset_path="D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test"):

    directory_contents = os.listdir(dataset_path)

    class_path_dict = {}

    for subdirectory in directory_contents:
        images = os.listdir(dataset_path + "\\" + subdirectory)

        for image in images:
            if image.endswith(".jpg"):
                full_image_path = dataset_path + "\\" + subdirectory + "\\" + image
                if subdirectory.lower() in class_path_dict:
                    class_path_dict[subdirectory.lower()].append(full_image_path)
                else:
                    class_path_dict[subdirectory.lower()] = [full_image_path]

    return class_path_dict

print(get_paths_to_photos_and_labels())

{'airplane': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Airplane\\2e89b6830d277669.jpg'], 'apple': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\44749496c016708d.jpg', 'D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\964b69442edeec04.jpg', 'D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\ba813e858a702b5a.jpg'], 'backpack': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Backpack\\723e70fe85e0da69.jpg'], 'banana': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Banana\\eeb93d366d6c69e7.jpg'], 'baseball bat': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Baseball bat\\9a6747129ee9d39d.jpg'], 'baseball glove': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Baseball glove\\0298c70279b6e842.jpg', 'D:\\AGH\\VI semestr\\ML\\datasetGoogle\

In [28]:

def get_all_images_metadata_dict(path="D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\csv_folder\\test-annotations-bbox.csv"):
    my_dict = {}
    with open(path, mode='r') as infile:
        reader = csv.reader(infile)
        for row in reader:
            if row[0] not in my_dict:
                my_dict[row[0]] = []
                my_dict[row[0]].append(row[1:])
            else:
                my_dict[row[0]].append(row[1:])

    return my_dict


In [29]:
def get_class_id_to_open_image_label_map(path = "D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\csv_folder\\class-descriptions-boxable.csv"):
    f = open(path)
    class_map = {}
    for line in f:
        data_line = line.rstrip().split(',')
        class_map[data_line[0]] = data_line[1]
    return class_map

In [65]:
def convert_open_image_label_to_coco(class_map):

    class_map = {value.lower():key for key, value in class_map.items()}

    class_map["street sign"] = class_map.pop("traffic sign")
    class_map["cow"] = class_map.pop("bull")
    class_map["eye glasses"] = class_map.pop("glasses")
    class_map["shoe"] = class_map.pop("boot")
    class_map["hair drier"] = class_map.pop("hair dryer")
    class_map["donut"] = class_map.pop("doughnut")
    class_map["microwave"] = class_map.pop("microwave oven")
    class_map["cell phone"] = class_map.pop("telephone")
    class_map["keyboard"] = class_map.pop("computer keyboard")
    class_map["remote"] = class_map.pop("remote control")
    class_map["mouse"] = class_map.pop("computer mouse")
    class_map["tv"] = class_map.pop("television")
    class_map["dining table"] = class_map.pop("kitchen dining room table")
    class_map["potted plant"] = class_map.pop("plant")
    class_map["skis"] = class_map.pop("ski")
    class_map["sports ball"] = class_map.pop("ball")
    class_map["cup"] = class_map.pop("coffee cup")
    class_map["frisbee"] = class_map.pop("flying disc")
    # hairbrush

    return {value:key for key, value in class_map.items()}



In [31]:
def get_image_labels(image_name):
    metadata = all_images_metadata_dict[image_name]
    readable = []
    for entry in metadata:
        readable.append(class_id_to_coco_label_map[entry[1]])

    return readable

In [32]:
def predict_single_image(path="D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Baseball glove\\0298c70279b6e842.jpg"):
    image_name = os.path.basename(path).split(".")[0]
    image = Image.open(path)
    result = predict(image)
    results = {key:value.numpy() for key,value in result.items()}
    # detection_classes = results['detection_classes']
    # print(results)
    # print("model thinks: " + coco[detection_classes[0][0]])
    # print("model thinks: " + coco[detection_classes[0][1]])
    # print("model thinks: " + coco[detection_classes[0][2]])
    # print("model thinks: " + coco[detection_classes[0][3]])
    # print("accually: ")
    # print(get_image_labels(image_name))
    return results, image_name

In [80]:
# 1.
#     bierzemy znane wszystkie klasy i boxy z openimage
# 2.
#     bierzemy znalezione klasy, boxy i scory tylko powyzej 0.5
# 3.
#     dla kazdego boxa z 1. liczymy wspólna/suma [PÓL] z każdym boxem z 2. => match if result >= 0.5
# 4.
#     jesli dla jednego boxa z 1 jest wiecej niz 1 z 2 to wbieriamy ten z najmocniejszym results
#     jesli dla jednego boxa z 2 jest wiecej niz 1 z 1 to wbieriamy ten z najmocniejszym results
# 5.
# ????????????????????????????????????
#     update confusion matrix
# 6. ToDo


def perform_analysis(model_result, image_name):
    labels_original = get_image_labels(image_name)
    print(all_images_metadata_dict[image_name][4])
    # print(labels_original)
    coco_labels_original = [i for i in labels_original if i in coco_list]
    print(labels_original)
    print(coco_labels_original)
    print(len(model_result['detection_classes'][0]))
    print(model_result)
    detected_classes = model_result['detection_classes'][0]
    detected_classes_probability = model_result['detection_scores'][0]

    most_important_detections = []

    # for d_class, score in zip(detected_classes,detected_classes_probability):
    #     if score >= 0.5 or len(most_important_detections) <= 2:



    print(detected_classes)
    print(detected_classes_probability)
    # readable_detected_classes = coco[]

In [66]:
# obtain all needed dicts

class_id_to_open_image_label_map = get_class_id_to_open_image_label_map()
class_id_to_coco_label_map =  convert_open_image_label_to_coco(class_id_to_open_image_label_map)

all_images_metadata_dict = get_all_images_metadata_dict()

In [58]:
coco = get_coco_labels()
coco_list  = list(coco.values())

# get_downloadable_labels_str(coco)
# copy returned string
# open in terminal
# python main.py downloader --classes <paste_here_your_string> --type_csv <validation/test/train> --limit XX

In [81]:
results, image_name = predict_single_image()
s = perform_analysis(results, image_name)

['freeform', '/m/03grzl', '1', '0.444979', '0.509261', '0.486219', '0.579589', '0', '0', '0', '0', '0']
['glove', 'glove', 'person', 'baseball bat', 'baseball glove', 'man', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'sports equipment', 'clothing', 'clothing', 'clothing', 'clothing', 'clothing', 'clothing', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'human arm', 'sports uniform', 'sports uniform', 'sports uniform']
['person', 'baseball bat', 'baseball glove']
100
{'detection_boxes': array([[[2.96614259e-01, 2.26320758e-01, 7.78682113e-01, 5.01484871e-01],
        [3.21174473e-01, 4.48274255e-01, 7.57149994e-01, 6.20503068e-01],
        [3.17986786e-0